In [47]:
import numpy as np
import math

In [48]:
def calculate_equivalence_classes(data, condition_attrs):
    data_indexed = data.set_index(condition_attrs)
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [49]:
def entropy(data,condition_feature,decision_attr):
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()
    return entropy_D

In [50]:
def calculate_contidion_entropy(data,condition_feature,decision_attr):
    total_records = len(data)
    equivalence_classes = calculate_equivalence_classes(data,condition_feature)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    entropy_DB = 0
    entropy_DB_sum=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB = -class_prob * decision_prob * math.log(decision_prob, 2)
                entropy_DB_sum=entropy_DB_sum+entropy_DB
    return entropy_DB_sum

In [51]:
def SGF(data,condition_feature,condition_feature_A,decision_attr):
    if len(condition_feature)==0:
        return entropy(data,condition_feature,decision_attr) - calculate_contidion_entropy(data,condition_feature_A,decision_attr)
    else:
        return calculate_contidion_entropy(data,condition_feature,decision_attr)-calculate_contidion_entropy(data,condition_feature_A,decision_attr)

In [52]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import pandas as pd

# entropy_DC=calculate_contidion_entropy(data,feature_columns,'target')
data = pd.read_csv('data/Soybean/soybean.csv')
feature_columns = [ col for col in data.columns if col not in ['id','target']]

target='target'
B=[]
w=0.1

while len(B)<len(feature_columns):
    S={}
    for col in feature_columns:
        if col not in B:
            C=B.copy()
            C.append(col)
            S[col] = SGF(data,B,C,target)
    max_key = max(S.items(), key=lambda x: x[1])[0]
    # print(max_key)
    B.append(max_key)
    if calculate_contidion_entropy(data,B,target)-calculate_contidion_entropy(data,feature_columns,target)<w:
        break
for col in B:
    D=B.copy()
    D.remove(col)
    if calculate_contidion_entropy(data,D,target)-calculate_contidion_entropy(data,feature_columns,target)<w:
        B.remove(col)

In [53]:
print(B)

['f23', 'f16', 'f2', 'f29', 'f4', 'f19', 'f8', 'f7']


In [54]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('data/Soybean/soybean.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id', 'target']]
Y = df["target"]
X = df[B]
def SVM(trainX, trainY, testX, testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=500)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY) / testY.shape[0]
    return precision

        # Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  UserWarning,


0.8549445865302643


In [55]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('data/Soybean/soybean.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
X = df[B]
Y = df['target']
dt = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.7261935208866155


In [56]:
from sklearn.neighbors import KNeighborsClassifier

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
df = pd.read_csv('data/Soybean/soybean.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
Y=df["target"]
X=df[B]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)


0.7466538789428816
